<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [5]:
!pip3 install beautifulsoup4
!pip3 install requests

In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [4]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

# If your notebook already defined static_url, we’ll use it; otherwise we fall back to the live page.
url = (globals().get("static_url")
       or "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches")
url = url.rstrip("?").strip()

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

resp = requests.get(url, headers=headers, timeout=30)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "html.parser")
print(soup.title.get_text(strip=True) if soup.title else "No <title>")


List of Falcon 9 and Falcon Heavy launches - Wikipedia


Create a `BeautifulSoup` object from the HTML `response`


In [5]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [6]:
(spacex_df['rocket.rocket_name'] == 'Falcon 9').sum()

NameError: name 'spacex_df' is not defined

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [ ]:
def _txt(el):
    return el.get_text(" ", strip=True) if el else None

def split_date_time(td):
    t = _txt(td) or ""
    parts = [p.strip() for p in t.split(",", 1)]
    if len(parts) == 2:
        return parts[0], parts[1]
    return t, None

def booster_version(td):
    t = _txt(td) or ""
    m = re.search(r"(Block\s*\d+|v\d+(?:\.\d+)?|FT)", t, flags=re.I)
    return m.group(0) if m else t

def landing_status(td):
    # keep first plain string (e.g., "ASDS landing", "Ocean", "Success", "Failure")
    t = _txt(td) or ""
    return re.sub(r"\[[^\]]*\]", "", t).strip()

def get_mass_kg(td):
    t = re.sub(r"\[[^\]]*\]", "", _txt(td) or "")
    m = re.search(r"([\d,]+(?:\.\d+)?)\s*kg", t, flags=re.I)
    return float(m.group(1).replace(",", "")) if m else None

def first_text_or_link(td):
    if td and td.a and td.a.string:
        return td.a.string.strip()
    return _txt(td)



## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [14]:
tables = soup.find_all("table")
# Falcon-9 launch records live in "wikitable plainrowheaders collapsible" blocks
launch_tables = soup.select("table.wikitable.plainrowheaders.collapsible")
if not launch_tables:
    # fallback: use the 3rd table on the page like many course snapshots do
    launch_tables = [tables[2]]

# Use the first launch table to get column headers
ths = launch_tables[0].find_all("th")
column_names = []
for th in ths:
    name = re.sub(r"\s+", " ", th.get_text(" ", strip=True))
    name = re.sub(r"\[[^\]]*\]", "", name).strip()
    if name:
        column_names.append(name)

print(column_names)


['Flight No.', 'Date and time ( UTC )', 'Version, Booster', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing', '1', '2', '3', '4', '5', '6', '7']


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [15]:
launch_dict = {
    "Flight No.": [],
    "Date": [],
    "Time": [],
    "Version Booster": [],
    "Launch site": [],
    "Payload": [],
    "Payload mass": [],
    "Orbit": [],
    "Customer": [],
    "Launch outcome": [],
    "Booster landing": [],
}

rows_parsed = 0

for tbl in launch_tables:
    for tr in tbl.find_all("tr"):
        th = tr.find("th")
        if not th:
            continue
        fn = (th.string or "").strip()
        if not fn.isdigit():
            continue

        tds = tr.find_all("td")
        if len(tds) < 9:
            continue

        d, tm = split_date_time(tds[0])
        bv = booster_version(tds[1])
        site = first_text_or_link(tds[2])
        payload = first_text_or_link(tds[3])
        mass = get_mass_kg(tds[4])
        orbit = first_text_or_link(tds[5])
        customer = first_text_or_link(tds[6])
        launch_outcome = (list(tds[7].stripped_strings) or [""])[0]
        booster_land = landing_status(tds[8])

        launch_dict["Flight No."].append(fn)
        launch_dict["Date"].append(d)
        launch_dict["Time"].append(tm)
        launch_dict["Version Booster"].append(bv)
        launch_dict["Launch site"].append(site)
        launch_dict["Payload"].append(payload)
        launch_dict["Payload mass"].append(mass)
        launch_dict["Orbit"].append(orbit)
        launch_dict["Customer"].append(customer)
        launch_dict["Launch outcome"].append(launch_outcome)
        launch_dict["Booster landing"].append(booster_land)

        rows_parsed += 1

rows_parsed


121

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [16]:
df = pd.DataFrame(launch_dict)

# Normalize
df["Flight No."] = pd.to_numeric(df["Flight No."], errors="coerce")
df["Payload mass"] = pd.to_numeric(df["Payload mass"], errors="coerce")

# Optional: make a single datetime (keep Date/Time columns too)
dt_str = (df["Date"].fillna("") + " " + df["Time"].fillna("")).str.strip()
df["Launch_UTC"] = pd.to_datetime(dt_str, errors="coerce", utc=True, infer_datetime_format=True)

df.head(10)


,Flight No.,Date,Time,Version Booster,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing,Launch_UTC
0,1,4 June 2010,18:45,v1.0,CCAFS,Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,Failure (parachute),2010-06-04 18:45:00+00:00
1,2,8 December 2010,15:43 [ 13 ],v1.0,CCAFS,Dragon,NaN,LEO,NASA,Success,Failure (parachute),NaT
2,3,22 May 2012,07:44 [ 17 ],v1.0,CCAFS,Dragon,525.0,LEO,NASA,Success,No attempt,NaT
3,4,8 October 2012,00:35 [ 21 ],v1.0,CCAFS,SpaceX CRS-1,4700.0,LEO,NASA,Success,No attempt,NaT
4,5,1 March 2013,15:10,v1.0,CCAFS,SpaceX CRS-2,4877.0,LEO,NASA,Success,No attempt,2013-03-01 15:10:00+00:00
5,6,29 September 2013,16:00 [ 30 ],v1.1,VAFB,CASSIOPE,500.0,Polar orbit,MDA,Success,Uncontrolled (ocean),NaT
6,7,3 December 2013,22:41 [ 34 ],v1.1,CCAFS,SES-8,3170.0,GTO,SES,Success,No attempt,NaT
7,8,6 January 2014,22:06 [ 41 ],v1.1,CCAFS,Thaicom 6,3325.0,GTO,Thaicom,Success,No attempt,NaT
8,9,18 April 2014,19:25 [ 21 ],v1.1,Cape Canaveral,SpaceX CRS-3,2296.0,LEO,NASA,Success,Controlled (ocean),NaT
9,10,14 July 2014,15:15,v1.1,Cape Canaveral,Orbcomm-OG2,1316.0,LEO,Orbcomm,Success,Controlled (ocean),2014-07-14 15:15:00+00:00


In [21]:
import pandas as pd, requests

# Try v3 first (has rocket.rocket_name). If that fails, fall back to v4.
try:
    launches = requests.get("https://api.spacexdata.com/v3/launches", timeout=30).json()
    spacex_df = pd.json_normalize(launches)
    spacex_df["rocket_name"] = spacex_df["rocket.rocket_name"]
except Exception:
    launches = requests.get("https://api.spacexdata.com/v4/launches", timeout=30).json()
    rockets = requests.get("https://api.spacexdata.com/v4/rockets", timeout=30).json()
    id2name = {r["id"]: r["name"] for r in rockets}
    spacex_df = pd.json_normalize(launches)
    spacex_df["rocket_name"] = spacex_df["rocket"].map(id2name)

# Q2: how many Falcon 9 launches (i.e., remove Falcon 1)
falcon9_count = (spacex_df["rocket_name"] == "Falcon 9").sum()
falcon9_count


103

In [23]:
import pandas as pd
import numpy as np

# 1) Find or derive a 'landpad' column
cand_cols = ["landingPad", "cores.0.landpad", "rocket.first_stage.cores.0.landpad"]
col = next((c for c in cand_cols if c in spacex_df.columns), None)

if col is None:
    if "cores" in spacex_df.columns:
        spacex_df["landpad"] = spacex_df["cores"].apply(
            lambda x: (x[0].get("landpad") if isinstance(x, list) and len(x) else None)
        )
        col = "landpad"
    elif "rocket.first_stage.cores" in spacex_df.columns:
        spacex_df["landpad"] = spacex_df["rocket.first_stage.cores"].apply(
            lambda x: (x[0].get("landpad") if isinstance(x, list) and len(x) else None)
        )
        col = "landpad"
    else:
        raise KeyError("No landpad-like column found. Inspect spacex_df.columns.")

# 2) Count missing
missing_landpad = spacex_df[col].isna().sum()
total_rows = len(spacex_df)
missing_landpad, total_rows


(111, 111)

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
